### Notes

- Evan used Smith-Waterman for original library generation. Let's stick with this for now.
- How much of an issue is speed? Need to compute this for several packages if biopython is slow for millions of pairs.
- adapted code from https://www.biostars.org/p/42687/ (old python 2 code. May not actually use)
- https://lh3.github.io/2018/11/25/on-the-definition-of-sequence-identity useful description of compressed id
- https://www.youtube.com/watch?v=oxHuUdYLA7E&t=587s&ab_channel=LanaCaldarevic

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

In [4]:
from Bio.Blast.Applications import NcbiblastpCommandline
#from StringIO import StringIO
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

In [14]:
s50k = pd.read_csv('../learn2therm_sample_50k_exploration.csv')
print(s50k.iloc[0])

Unnamed: 0                                                                               0
local_gap_compressed_percent_id                                                   0.287582
scaled_local_query_percent_id                                                     0.217822
scaled_local_symmetric_percent_id                                                 0.215686
query_align_len                                                                        160
query_align_cov                                                                   0.792079
subject_align_len                                                                      152
subject_align_cov                                                                 0.737864
bit_score                                                                              131
thermo_index                                                                           875
meso_index                                                                           12897

In [20]:
metrics = ['local_gap_compressed_percent_id', 'scaled_local_query_percent_id','scaled_local_symmetric_percent_id',
           'query_align_len','query_align_cov','subject_align_len','subject_align_cov','bit_score']

In [21]:
print(s50k.iloc[0][metrics])

local_gap_compressed_percent_id      0.287582
scaled_local_query_percent_id        0.217822
scaled_local_symmetric_percent_id    0.215686
query_align_len                           160
query_align_cov                      0.792079
subject_align_len                         152
subject_align_cov                    0.737864
bit_score                                 131
Name: 0, dtype: object


In [6]:
seq_s1 = s50k.iloc[0][['m_protein_seq','t_protein_seq']]

In [8]:
seq_s1[0]

'MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLYDLLAPRVYGLIRRVLRDPALAEEVTQEVLVEVWRRAARFDPAQGSANAWVFTIAHRRAVDRVRAEQKAADRTVRAGAAALDSPYDSVADEVSGRLERRQVRHCLDALTGLQREVVTLAYYQGHSYPQVAELLKTPLGTVKTRMRDGLIRLRDCLGVEATA'

In [31]:
min(len(seq_s1[0]), len(seq_s1[1]))

202

In [13]:
# # Create two sequence files
# seq1 = SeqRecord(Seq(seq_s1[0]), id="seq1")
# seq2 = SeqRecord(Seq(seq_s1[1]), id="seq2")

# SeqIO.write(seq1, "seq1.fasta", "fasta")
# SeqIO.write(seq2, "seq2.fasta", "fasta")

# # Run BLAST and parse the output as XML
# output = NcbiblastpCommandline(query="seq1.fasta", subject="seq2.fasta", outfmt=5)()[0]
# blast_result_record = NCBIXML.read(StringIO(output))

# # Print some information on the result
# for alignment in blast_result_record.alignments:
#     for hsp in alignment.hsps:
#         print('****Alignment****')
#         print('sequence:', alignment.title)
#         print('length:', alignment.length)
#         print('e value:', hsp.expect)
#         print(hsp.query)
#         print(hsp.match)
#         print(hsp.sbjct)

In [34]:
# https://biopython.org/docs/1.75/api/Bio.pairwise2.html
from Bio import pairwise2
from Bio.Align import substitution_matrices

matrix = substitution_matrices.load('BLOSUM62')
alignments = pairwise2.align.localds(seq_s1[0], seq_s1[1], match_dict = matrix, open = -11, extend = -1)

for alignment in alignments:
    print(format_alignment(*alignment))

49 LAPRVYGLIRRVLRDPALAEEVTQEVLVEVWRRAARFDPAQGSANAWVFTIAHRRAVDRVRAEQK---------AADRTVRAGAAALDSPYDSVADEVSGRLERRQVRHCLDALTGLQREVVTLAYYQGHSYPQVAELLKTPLGTVKTRMRDGLIRLRDCL
   |....|....|..|.||.||...||.........|.|.... ...||...|.........|..|.         ..|...............|...|....|....|...|..|....|..|.||...|.||...||...||.|||..|...|..|||..|
24 LLDQLYSAALRMTRNPADAEDLVQETYLKAYNAFASFKEGT-NLKAWLYRILTNTYINSYRKKQRQPLQQPAEDITDWQLAQAESHTSRGLRSAEVEALEQLPDADVKDALQRLPEDFRLAVYLADVEGFSYKEIAEIMGTPIGTVMSRLHRGRGRLRELL
  Score=142

49 LAPRVYGLIRRVLRDPALAEEVTQEVLVEVWRRAARFDPAQGSANAWVFTIAHRRAVDRVRAEQKA---------ADRTVRAGAAALDSPYDSVADEVSGRLERRQVRHCLDALTGLQREVVTLAYYQGHSYPQVAELLKTPLGTVKTRMRDGLIRLRDCL
   |....|....|..|.||.||...||.........|.|.... ...||...|.........|..|..         .|...............|...|....|....|...|..|....|..|.||...|.||...||...||.|||..|...|..|||..|
24 LLDQLYSAALRMTRNPADAEDLVQETYLKAYNAFASFKEGT-NLKAWLYRILTNTYINSYRKKQRQPLQQPAEDITDWQLAQAESHTSRGLRSAEVEALEQLPDADVKDALQRLPEDFRLAVYLADVEGFSYKEIAEIMGTPIGTVMSRLHRGRGRLRE

In [35]:
alignments[0]

Alignment(seqA='MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLYDLLAPRVYGLIRRVLRDPALAEEVTQEVLVEVWRRAARFDPAQGSANAWVFTIAHRRAVDRVRAEQK---------AADRTVRAGAAALDSPYDSVADEVSGRLERRQVRHCLDALTGLQREVVTLAYYQGHSYPQVAELLKTPLGTVKTRMRDGLIRLRDCLGVEATA-------------', seqB='-------------------------MPSQITESERIELAERFERDALPLLDQLYSAALRMTRNPADAEDLVQETYLKAYNAFASFKEGT-NLKAWLYRILTNTYINSYRKKQRQPLQQPAEDITDWQLAQAESHTSRGLRSAEVEALEQLPDADVKDALQRLPEDFRLAVYLADVEGFSYKEIAEIMGTPIGTVMSRLHRGRGRLRELLADVARERGFIKQGVEVAKR', score=142.0, start=48, end=209)

In [33]:
142/202

0.7029702970297029